In [ ]:
import tensorflow as tf
import utils
import tensorflow_addons as tfa
from data_parameters import data_param

print(tf.config.experimental.get_visible_devices())

per_sys = data_param['per_sys']
N_elem = len(per_sys)
spectrum_length = data_param['spectrum_length']
max_buffer = data_param['max_buffer']
batch_size = 32


val_dataset = utils.get_dataset(tf.data.TFRecordDataset.list_files('Core-loss EELS TFRecord/validationset/VALIDATION*.tfrecords', shuffle=True))
val_dataset = val_dataset.shuffle(buffer_size= max_buffer , reshuffle_each_iteration=True).batch(batch_size).prefetch(tf.data.AUTOTUNE)  


In [ ]:
model1 = tf.keras.models.load_model('trained element identification models/UNet',custom_objects={'custom_loss': utils.custom_loss}) 
model2 = tf.keras.models.load_model('trained element identification models/ViT',custom_objects={'custom_loss': utils.custom_loss})

models = [model1,model2]

for idx,model in enumerate(models):
        model._name = model.name + f'_{idx}'

model_input = tf.keras.Input(shape=(spectrum_length, 1))
model_outputs = [model(model_input) for model in models]
ensemble_output = tf.keras.layers.Average()(model_outputs)
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)

In [ ]:
ensemble_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),   #doesn't matter here, no training
                       loss=utils.custom_loss,                                   #doesn't matter here, no training
                       metrics= [tfa.metrics.F1Score(N_elem, 'weighted', 0.8, name = 'f1')]) 

In [ ]:
ensemble_model.evaluate(val_dataset)

In [ ]:
ensemble_model.save(f'new_trained_models/ensemble')